In [1]:
!pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.8 MB/s 


# Conjunto de datos personalizado

En situaciones en las que los datos no están fácilmente disponibles pero son necesarios, **tendrá que recurrir a recopilar los datos usted mismo.** Hay muchos métodos que puede usar para adquirir estos datos, desde webscraping hasta API. Pero a veces, terminará necesitando crear **datos falsos o "ficticios".** Los datos ficticios pueden ser útiles en momentos en los que conoce las funciones exactas que utilizará y los tipos de datos incluidos, pero simplemente no tiene los datos en sí.

Aquí, le mostraré **cómo creé 100 000 filas de datos ficticios.** Estos datos tampoco son puramente aleatorios. Si lo fuera, construirlo habría sido mucho más fácil. A veces, al crear datos ficticios desde cero, deberá desarrollar tendencias o patrones dentro de los datos que puedan reflejar el comportamiento del mundo real. 

# **La necesidad de construir un conjunto de datos**

**Supongamos que está creando una aplicación desde cero y necesita establecer una gran base de usuarios para, por ejemplo, realizar pruebas.** Se le proporciona una lista de características y sus respectivos tipos de datos.

**Esta base de usuarios también debe reflejar con cierta precisión los usuarios y las tendencias del mundo real, por lo que no puede ser completamente aleatoria.** Por ejemplo, no desea que un usuario tenga un título universitario y también tenga 10 años. Además, es posible que no desee una sobrerrepresentación de un punto de datos específico, como más hombres que mujeres. Todos estos son puntos a tener en cuenta a la hora de crear su conjunto de datos.

**Dado que los datos del mundo real rara vez son verdaderamente aleatorios, un conjunto de datos como este sería una excelente simulación para otros conjuntos de datos que manejará en el futuro.**

# Construcción del conjunto de datos

Para codificar, comience importando las siguientes bibliotecas:

In [2]:
import pandas as pd
import uuid
import random
from faker import Faker
import datetime

**Size**

El tamaño del conjunto de datos será de 100 000 puntos de datos (puede hacer más, pero el procesamiento puede demorar más). Asigné esta cantidad a una variable constante, que usé en todo momento:

In [3]:
num_afiliados = 5000

**Características**

Elegí 10 características que esperaba que fueran las más comunes en un conjunto de datos regular de usuarios. Estas características y los respectivos tipos de datos son:

     - ID: una cadena única de caracteres para identificar a cada usuario.
     - Gender (Sexo:) tipo de datos de cadena de tres opciones.
     - Name (Nombre:) tipo de datos de cadena del nombre y apellido del usuario.
     - Email (Correo electrónico:) q  tipo de datos de cadena de la dirección de correo electrónico del usuario.
     - Date of Birth (Fecha de nacimiento:) formato de cadena de año-mes-día.


Ingresé lo anterior como una lista de características para inicializar un marco de datos de Pandas:

In [4]:
# A list of 10 features
features = [
    "id_afiliado",
    "genero_afiliado",
    "name",
    "email",
    "dob"
]# Creating a DF for these features
df = pd.DataFrame(columns=features)

**Creación de datos**

Algunos atributos anteriores normalmente deberían contener datos desequilibrados. Se puede asumir con seguridad con una investigación rápida, algunas opciones no estarán igualmente representadas. Para un conjunto de datos más realista, estas tendencias deben reflejarse.

**IDs**

Para el atributo ID, utilicé la biblioteca uuid para generar una cadena aleatoria de caracteres 100 000 veces. Luego, lo asigné al atributo ID en el marco de datos.

In [5]:
df['id_afiliado'] = [uuid.uuid4().hex for i in range(num_afiliados)]

**UUID es una gran biblioteca para generar identificaciones únicas para cada usuario debido a su posibilidad astronómicamente baja de duplicar una identificación.** Es una gran opción cuando se trata de generar conjuntos de caracteres de identificación únicos. Pero, si desea asegurarse de que no se repitieron las ID, puede realizar una verificación simple en el marco de datos con lo siguiente:

In [6]:
print(df['id_afiliado'].nunique()==num_afiliados)

True


Esto devolverá True si todas las ID en el conjunto de datos son únicas.

**Gender**

**Este atributo es uno de los casos en los que probablemente no se debería utilizar una elección igualmente aleatoria. Porque se puede suponer con seguridad que cada elección no tiene la misma probabilidad de ocurrir.**

Para el género, proporcioné tres opciones: masculino, femenino y na. Sin embargo, si tuviera que usar la biblioteca aleatoria de Python, cada opción podría mostrarse igualmente en el conjunto de datos. En realidad, habría significativamente más opciones masculinas y femeninas que na. Así que decidí mostrar ese desequilibrio en los datos:

In [7]:
genders = ["male", "female", "na"]

In [8]:
df['genero_afiliado'] = random.choices(
    genders, 
    weights=(47,47,6), 
    k=num_afiliados
)

**Name**

Aquí usé la biblioteca Faker para crear miles de nombres para todos estos usuarios. **La biblioteca Faker es excelente en esta situación porque tiene una opción para nombres masculinos y femeninos.**}} Para procesar los nombres de género, creé una función para asignar nombres en función de un género determinado.

In [9]:
# Instantiating faker
faker = Faker()

def name_gen(gender):
    """
    Quickly generates a name based on gender
    """
    if gender=='male':
        return faker.name_male()
    elif gender=='female':
        return faker.name_female()
    
    return faker.name()# Generating names for each user
df['name'] = [name_gen(i) for i in df['genero_afiliado']]

Usé mi función simple para producir rápidamente una lista de nombres basada en los datos del atributo "Género" antes y la asigné al marco de datos.

Al usar la biblioteca aleatoria, proporcioné la función de opciones/choices () con la lista de opciones de género, pesos para cada opción y cuántas opciones hacer representadas por "k". Luego se asignó al atributo de "género" del marco de datos. El desequilibrio que describí antes está representado en la sección de ponderaciones con una opción "na" que aparece aproximadamente el 6% de las veces.

Al igual que "Género" antes, usé random.choices() pero sin pesos porque este atributo se puede dividir aleatoriamente entre las dos opciones.

**Email**

El correo electrónico resultó ser uno de los atributos más complicados. Quería crear direcciones de correo electrónico relacionadas con los nombres generados. Sin embargo, probablemente habría una posibilidad de duplicación porque las personas pueden compartir el mismo nombre pero no el mismo correo electrónico.

Primero, creé una nueva función que daría formato a los nombres en direcciones de correo electrónico con un nombre de dominio predeterminado. También manejaría direcciones duplicadas simplemente agregando un número aleatorio al final del nombre formateado:

In [10]:
def emailGen(name, duplicateFound=False):
    """
    Generates a random email address based on the given name. 
    Adds a number at the end if a duplicate address was found.
    """
    # Fake domain name to use
    dom = "@gmail.com"
    
    # Lowercasing and splitting
    name = name.lower().split(" ")
    
    # Random character to insert in the name
    chars = [".", "_"]
    
    new_name = name[0] + random.choice(chars) + name[1] 
    
    # Further distinguishing the email if a duplicate was found
    if duplicateFound:
        
        # Random number to insert at the end
        num = random.randint(0,100)
        
        # Inserting at the end
        new_name = new_name + str(num)
        
    # Returning the email address with the domain name attached
    return new_name + dom

Ahora, para aprovechar adecuadamente el propósito de esta función, creé un ciclo que volvería a ejecutar la función cuando fuera necesario mientras iteraba a través del atributo "Nombre". El ciclo seguiría volviendo a ejecutar la función hasta que se creara un nombre de correo electrónico único.

In [11]:
emails = []

for name in df['name']:
    
    # Generating the email
    email = emailGen(name)
    
    # Looping until a unique email is generated
    while email in emails:
        
        # Creating an email with a random number
        email = emailGen(name, duplicateFound=True)
    
    # Attaching the new email to the list
    emails.append(email)
    
df['email'] = emails

Después de generar todos los correos electrónicos, los asigné al atributo "Correo electrónico" del marco de datos. También puede hacer una verificación opcional para ver que cada correo electrónico sea único con el mismo método que las ID.

**Date of Birth**

Este atributo es fácil ya que es similar a "Último inicio de sesión". Todo lo que hice fue cambiar el formato de la hora eliminando la hora, los minutos y los segundos. Usé datetime nuevamente para ayudar a elegir aleatoriamente una fecha para cada usuario, pero esta vez el rango de tiempo comenzó desde 1980 hasta 2006 para obtener una buena distribución aleatoria de edades. El siguiente código es prácticamente el mismo que antes, pero con un formato y un intervalo de fechas diferentes:

In [12]:
def random_dob(start, end, n):
    """
    Generating a list of a set number of timestamps
    """
    
    # The timestamp format
    frmt = "%Y-%m-%d"
    
    # Formatting the two time periods
    stime = datetime.datetime.strptime(start, frmt)
    etime = datetime.datetime.strptime(end, frmt)
    
    # Creating the pool for random times
    td = etime - stime
    
    # Generating a list with the random times
    times = [(random.random() * td + stime).strftime(frmt) for _ in range(n)]
    
    return times

df['dob'] = random_dob("1962-01-01", "2004-01-01", num_afiliados)

**Saving the Dataset**

Ahora que los datos están completos y si estaba codificando, siéntase libre de ver el marco de datos antes de decidir guardarlo. Si todo se ve bien, guarde el marco de datos como un archivo .csv con este simple comando:

In [13]:
df.to_csv('dataset_afiliados.csv')